In [ ]:
from loader import Loader, Loader2
import torch
import torchvision.transforms as transforms
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
import numpy as np
import model

In [ ]:
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
latent_dim=32

In [ ]:
def get_plabels_by_vae(vae, discriminator,samples,cycle):
    # sample1k = []
    sub5k = Loader2(is_train=False,  transform=transform_test, path_list=samples)
    ploader = torch.utils.data.DataLoader(sub5k, batch_size=1, shuffle=False, num_workers=2)

    unlabel_scores = []
    vae.eval()
    discriminator.eval()
    with torch.no_grad():
        for idx, (inputs, targets) in enumerate(ploader):
            inputs, targets = inputs.to(device), targets.to(device)
            _, _, mu, _ = vae(inputs)
            preds = discriminator(mu)
            # preds = preds.cpu().data
            unlabel_scores.append(preds.view(preds.size(0)).item())
            # progress_bar(idx, len(ploader))
    idx=np.argsort(unlabel_scores)
    samples=np.array(samples)
    return samples[idx[-1000:]]

In [ ]:
vae = model.VAE(latent_dim)
discriminator = model.Discriminator(latent_dim)
vae=vae.to(device)
discriminator=discriminator.to(device)

In [ ]:
with open(f'./loss/batch_{1}.txt', 'r') as f:
    samples = f.readlines()

In [ ]:
sub5k = Loader2(is_train=False,  transform=transform_test, path_list=samples)
ploader = torch.utils.data.DataLoader(sub5k, batch_size=1, shuffle=False, num_workers=2)

unlabel_scores = []
vae.eval()
discriminator.eval()
with torch.no_grad():
    for idx, (inputs, targets) in enumerate(ploader):
        inputs, targets = inputs.to(device), targets.to(device)
        _, _, mu, _ = vae(inputs)
        preds = discriminator(mu)
        # preds = preds.cpu().data
        print(preds)
        unlabel_scores.append(preds.view(preds.size(0)).item())
        # progress_bar(idx, len(ploader))

In [ ]:
idx=np.argsort(unlabel_scores)
samples=np.array(samples)
samples[idx[-1000:]]

In [ ]:
np.sort(unlabel_scores)

In [ ]:
torch.topk(torch.from_numpy(np.array(unlabel_scores)),10)

In [ ]:
np.sort(unlabel_scores)[-10:]

In [ ]:
samples1k=get_plabels_by_vae(vae, discriminator,samples,cycle=2)

In [ ]:
cityspace_dir='/home/dang.hong.thanh/datasets/Cityspaces'
# image_transform=transforms.Compose([
#     # you can add other transformations in this list
#     transforms.ToTensor()])
# target_transform = transforms.Compose([
#     transforms.PILToTensor()
# ])
interp = nn.Upsample(size=(50, 50), mode='bilinear')


In [7]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
count_parameters(net)

## calucate uncertainty

In [ ]:
with open(f'./loss_seg/batch_0.txt', 'r') as f:
            samples = f.readlines()
samples=[sample[:-1] for sample in samples] # remove newline character

In [ ]:
samples[0]

In [ ]:
# pth='/home/dang.hong.thanh/PT4AL/checkpoint/ckpt.pth'
pth='/home/dang.hong.thanh/PT4AL/checkpoint_seg/main_0.pth'
checkpoint=torch.load(pth)
net.load_state_dict(checkpoint['net'])

In [ ]:
class Args:
    resize = 512
    base_size = 512
    crop_size = 50
    multi_scale= None

In [ ]:
args=Args()

In [ ]:
import numpy as np

In [ ]:
# def get_seg_plabels_confidence(net, samples, cycle):
    # dictionary with 10 keys as class labels
    # class_dict = {}
    # [class_dict.setdefault(x,[]) for x in range(10)]
device='cuda'
sample1k = []
subset = ActiveCityscapesSegmentation(args,cityspace_dir,split='train',files=samples)
subset=CityscapesSegmentation(args,cityspace_dir,split='val')
ploader = torch.utils.data.DataLoader(subset, batch_size=1, shuffle=False, num_workers=1)

uncertainty_scores = []
net.eval()
with torch.no_grad():
    for idx, samples in enumerate(ploader):
        inputs,targets=samples['image'],samples['label']
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = net(inputs)
        outputs=interp(outputs)
        # scores, predicted = outputs.max(1)
        # save top1 confidence score 
        # outputs = F.normalize(outputs, dim=1)
        probs = F.softmax(outputs, dim=1)
        # top1_scores.append(probs[0][predicted.item()])
        # top1_scores.append(probs[0][predicted.item()].cpu().item())
        # max_probs,indices=torch.max(probs,dim=1)
        # # print(outputs.size())
        # print(max_probs)
        # print(indices)
        # measures=torch.logical_and(probs>=0.35,probs<=0.75)
        best_probs,_=torch.max(probs,dim=1)
        measures=torch.lt(best_probs,0.5)    
        uncertainty_score=torch.sum(measures,dim=(-1,-2))
        # print(uncertainty_score)
        uncertainty_scores.append(uncertainty_score.cpu().item())
        # progress_bar(idx, len(ploader))

In [ ]:
samples=subset[0]
inputs,targets=samples['image'],samples['label']
inputs, targets = inputs.to(device), targets.to(device)

In [ ]:
inputs.shape

In [ ]:
outputs = net(inputs.unsqueeze(0))
outputs=interp(outputs)
scores, predicted = outputs.max(1)
print(scores)

In [ ]:
probs = F.softmax(outputs, dim=1)
print(probs)

In [ ]:
n_outputs = F.normalize(outputs, dim=1)
scores, predicted = n_outputs.max(1)
print(scores)
probs = F.softmax(n_outputs, dim=1)
print(probs)

In [ ]:
max_probs,indices=torch.max(probs,dim=1)
# print(outputs.size())
print(max_probs)

In [ ]:
max_probs,indices=torch.topk(probs,k=2,dim=1)
print(max_probs)

In [ ]:
max_probs.shape

In [ ]:
max_logits, _=torch.topk(outputs,dim=1,k=2)
print(max_logits)

In [ ]:
probs = F.softmax(outputs, dim=1)
print(probs)

In [ ]:
max_probs,indices=torch.max(probs,dim=1)
# print(outputs.size())
print(max_probs)

In [ ]:
max_probs.size()

In [ ]:
measures=torch.lt(max_probs,0.5)

In [ ]:
measures.size()

In [ ]:
print(torch.sum(measures,(-1,-2)))

In [ ]:
print(torch.sum(measures))

In [ ]:
238/(50*50)

In [ ]:
uncertainty_scores = []

In [ ]:
uncertainty_score=torch.sum(measures,(-1,-2))
uncertainty_scores.append(uncertainty_score.cpu().item())

In [ ]:
uncertainty_score

In [106]:
from mmseg.core.evaluation.metrics import eval_metrics
results = []
gt_seg_maps=[]
model.eval()
device='cuda'
with torch.no_grad():
    for batch_idx, samples in enumerate(testloader):
        # inputs,targets=samples['image'],samples['label']
        # inputs,targets=samples['img'].data[0],samples['gt_semantic_seg'].data[0]
        inputs,targets=samples['img'][0],samples['gt_semantic_seg'][0]
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        # data=dict()
        # data['img']=samples['img'].data
        # # data['gt_semantic_seg']=samples['gt_semantic_seg'].data
        # data['img_metas']=samples['img_metas'].data
        # outputs=model(return_loss=False,**samples)
        # print(outputs.size())
        # outputs=val_interp(outputs)
        # (_,H,W)=targets.size().data
        # outputs=nn.Upsample(size=(targets.size()[-2],targets.size()[-1]), mode='bilinear', align_corners=True)(outputs)
        _, predicted = outputs.max(1)
        # print(predicted.size())
        # print(targets[0].size())
        results.append(predicted.squeeze(0).cpu().numpy())
        gt_seg_maps.append(targets[0].squeeze(0).cpu().numpy())
ret_metrics=eval_metrics(results,gt_seg_maps,num_classes=19,ignore_index=225,metrics=['mIoU','mDice'],reduce_zero_label=False)
# print(ret_metrics['Dice'].mean())


In [104]:
print(ret_metrics['IoU'].mean())
print(ret_metrics['Dice'].mean())

0.6828102191699298
0.8041502137207482


## Evaluate by mmseg data and mmseg model

### Build dataloader

In [7]:
dataset_type = 'CityscapesDataset'
data_root='/home/dang.hong.thanh/datasets/Cityspaces'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
# test_pipeline = [
#     dict(type='LoadImageFromFile'),
#     dict(
#         type='MultiScaleFlipAug',
#         img_scale=(2048, 1024),
#         # img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
#         flip=False,
#         transforms=[
#             dict(type='Resize', keep_ratio=True),
#             dict(type='RandomFlip'),
#             dict(type='Normalize', **img_norm_cfg),
#             dict(type='ImageToTensor', keys=['img']),
#             dict(type='Collect', keys=['img']),
#         ])
# ]
# crop_size = (512, 1024)
# train_pipeline = [
#     dict(type='LoadImageFromFile'),
#     dict(type='LoadAnnotations'),
#     dict(type='Resize', img_scale=(2048, 1024), ratio_range=(0.5, 2.0)),
#     dict(type='RandomCrop', crop_size=crop_size, cat_max_ratio=0.75),
#     dict(type='RandomFlip', prob=0.5),
#     dict(type='PhotoMetricDistortion'),
#     dict(type='Normalize', **img_norm_cfg),
#     dict(type='Pad', size=crop_size, pad_val=0, seg_pad_val=255),
#     dict(type='DefaultFormatBundle'),
#     dict(type='Collect', keys=['img', 'gt_semantic_seg']),
# ]
val_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(2048, 1024),
        # img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img','gt_semantic_seg']),
        ]),
        # dict(type='Collect', keys=['img','gt_semantic_seg'])
]
val_cfg=dict(
        type=dataset_type,
        data_root=data_root,
        img_dir='images/val',
        ann_dir='gtFine/val',
        pipeline=val_pipeline,
        img_suffix='',
        seg_map_suffix='',
        split='/home/dang.hong.thanh/PT4AL/loss_seg/batch_0.txt',)

In [8]:
from mmseg.datasets import build_dataset
val_dataset=build_dataset(val_cfg)
print(val_dataset[0]['img'][0].size())
print(val_dataset[0]['gt_semantic_seg'][0].shape)

2023-02-27 05:20:34,585 - mmseg - INFO - Loaded 297 images



torch.Size([3, 1024, 2048])
(1024, 2048, 3)


In [50]:
from mmcv.parallel import collate
from functools import partial
import torch
testloader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=2,drop_last=False,
    collate_fn=partial(collate, samples_per_gpu=2))

### Buld model

In [22]:
from mmseg.apis import init_segmentor
config_file = '/home/dang.hong.thanh/mmsegmentation/configs/pspnet/pspnet_r18-d8_512x1024_80k_cityscapes.py'
checkpoint_file = 'pspnet_r18-d8_512x1024_80k_cityscapes_20201225_021458-09ffa746.pth'
model =init_segmentor(config_file, checkpoint_file, device='cuda')

load checkpoint from local path: pspnet_r18-d8_512x1024_80k_cityscapes_20201225_021458-09ffa746.pth


/home/dang.hong.thanh/PT4AL/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


### Testing

In [53]:
from mmseg.core.evaluation.metrics import intersect_and_union
from mmcv.parallel import DataContainer
from tqdm import tqdm
model.eval()
device='cuda'
pre_eval_results = []
with torch.no_grad():
    for batch_idx, data in tqdm(enumerate(testloader),ncols = 100,
               desc ="Tesing"):
        # inputs,targets=samples['image'],samples['label']
        # inputs,targets=samples['img'].data[0],samples['gt_semantic_seg'].data[0]
        # targets=data['gt_semantic_seg'][0]
        # targets = targets.to(device)
        input_data=dict()
        input_data['img_metas']=data['img_metas'][0].data
        input_data['img']=[data['img'][0].data.to(device)]
        outputs = model(return_loss=False, **input_data)
        # outputs=val_interp(outputs)
        # (_,H,W)=targets.size().data
        # outputs=nn.Upsample(size=(targets.size()[-2],targets.size()[-1]), mode='bilinear', align_corners=True)(outputs)
        # _, predicted = outputs[0].max(1)
        pred=outputs[0]
        seg_map=data['gt_semantic_seg'][0].cpu().squeeze().numpy()
        pre_eval_result=intersect_and_union(pred,seg_map,num_classes=19,ignore_index=255,label_map=dict(),reduce_zero_label=False)
        pre_eval_results.append(pre_eval_result)

Tesing: 500it [00:24, 20.61it/s]


In [52]:
data['gt_semantic_seg'][0].shape

torch.Size([1, 1024, 2048])

In [47]:
from mmseg.core import pre_eval_to_metrics
ret_metrics=pre_eval_to_metrics(pre_eval_results,metrics=['mIoU'])
print(ret_metrics['IoU'].mean())

0.74865


### Testing

## Redefind model

### Build model

In [60]:
device='cuda:0'
norm_cfg = dict(type='BN', requires_grad=True)
model_cfg = dict(
    type='SunSegmentor',
    backbone=dict(
        type='ResNetV1c',
        pretrained='open-mmlab://resnet18_v1c',
        depth=18,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        dilations=(1, 1, 2, 4),
        strides=(1, 2, 1, 1),
        norm_cfg=norm_cfg,
        norm_eval=False,
        style='pytorch',
        contract_dilation=True),
    decode_head=dict(
        type='PSPHead',
        in_channels=512,
        in_index=3,
        channels=128,
        pool_scales=(1, 2, 3, 6),
        dropout_ratio=0.1,
        num_classes=19,
        norm_cfg=norm_cfg,
        align_corners=False,
        loss_decode=dict(
            type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0)),
    # auxiliary_head=dict(
    #     type='FCNHead',
    #     in_channels=256,
    #     in_index=2,
    #     channels=64,
    #     num_convs=1,
    #     concat_input=False,
    #     dropout_ratio=0.1,
    #     num_classes=19,
    #     norm_cfg=norm_cfg,
    #     align_corners=True,
    #     loss_decode=dict(
    #         type='CrossEntropyLoss', use_sigmoid=False, loss_weight=0.4)),
    # model training and testing settings
    train_cfg=dict(),
    # test_cfg=dict(mode='slide', crop_size=(769, 769), stride=(513, 513))
    # test_cfg=None
    )

In [61]:
from mmseg.models.builder import build_segmentor
model = build_segmentor(model_cfg)
model.init_weights()
model = model.to(device)
import torch
# checkpoint=torch.load('/home/dang.hong.thanh/PT4AL/pspnet_r18-d8_512x1024_80k_cityscapes_20201225_021458-09ffa746.pth')
# model.load_state_dict(checkpoint['state_dict'],strict=True)
checkpoint1=torch.load('/home/dang.hong.thanh/PT4AL/checkpoint/ckpt.pth')
model.load_state_dict(checkpoint1['net'],strict=True)

/home/dang.hong.thanh/PT4AL/mmseg/models/backbones/resnet.py:431: UserWarning: DeprecationWarning: pretrained is a deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is a deprecated, '
/home/dang.hong.thanh/PT4AL/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(
2023-02-24 01:45:36,244 - mmseg - INFO - initialize ResNetV1c with init_cfg {'type': 'Pretrained', 'checkpoint': 'open-mmlab://resnet18_v1c'}
2023-02-24 01:45:36,244 - mmcv - INFO - load model from: open-mmlab://resnet18_v1c
2023-02-24 01:45:36,244 - mmcv - INFO - load checkpoint from openmmlab path: open-mmlab://resnet18_v1c
2023-02-24 01:45:36,256 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: f

<All keys matched successfully>

### TESTING

In [62]:
from mmseg.core.evaluation.metrics import intersect_and_union
from tqdm import tqdm
model.eval()
device='cuda'
pre_eval_results = []
with torch.no_grad():
    for batch_idx, samples in tqdm(enumerate(testloader),ncols = 100,
               desc ="Tesing"):
        # inputs,targets=samples['image'],samples['label']
        # inputs,targets=samples['img'].data[0],samples['gt_semantic_seg'].data[0]
        inputs,targets=samples['img'][0],samples['gt_semantic_seg'][0]
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        # outputs=val_interp(outputs)
        # (_,H,W)=targets.size().data
        # outputs=nn.Upsample(size=(targets.size()[-2],targets.size()[-1]), mode='bilinear', align_corners=True)(outputs)
        _, predicted = outputs.max(1)
        pred=predicted.squeeze(0).cpu().numpy()
        seg_map=targets.squeeze(0).cpu().numpy()
        pre_eval_result=intersect_and_union(pred,seg_map,num_classes=19,ignore_index=255,label_map=dict(),reduce_zero_label=False)
        pre_eval_results.append(pre_eval_result)


from mmseg.core import pre_eval_to_metrics
ret_metrics=pre_eval_to_metrics(pre_eval_results,metrics=['mIoU'])
print(ret_metrics['IoU'].mean())

Tesing: 0it [00:00, ?it/s]


KeyError: 'img'

## Redefine dataloader

In [63]:
CITYSCAPES_DIR='/home/dang.hong.thanh/datasets/Cityspaces'
class Args:
    a='a'
args=Args
args.resize = 512
args.base_size = 512
# args.crop_size = 321
args.crop_size = 512
args.batch_size=1
args.resume=False
from dataloaders.datasets.cityscapes import CityscapesSegmentation
import torch
val_dataset=CityscapesSegmentation(args,root=CITYSCAPES_DIR,split='val')
testloader = torch.utils.data.DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False, num_workers=2,drop_last=False)
# import torch.nn as nn
# val_interp = nn.Upsample(size=(args.crop_size, args.crop_size*2), mode='bilinear', align_corners=True)

Found 500 val images


### Testing

In [64]:
from mmseg.core.evaluation.metrics import intersect_and_union
from tqdm import tqdm
model.eval()
device='cuda'
pre_eval_results = []
with torch.no_grad():
    for batch_idx, samples in tqdm(enumerate(testloader),ncols = 100,
               desc ="Tesing"):
        inputs,targets=samples['image'],samples['label']
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        # print(outputs.size())
        # outputs=val_interp(outputs)
        # _,H,W=targets.size()
        # outputs=nn.Upsample(size=(H,W), mode='bilinear', align_corners=True)(outputs)
        _, predicted = outputs.max(1)
        pred=predicted.squeeze(0).cpu().numpy()
        seg_map=targets.squeeze(0).cpu().numpy()
        pre_eval_result=intersect_and_union(pred,seg_map,num_classes=19,ignore_index=255,label_map=dict(),reduce_zero_label=False)
        pre_eval_results.append(pre_eval_result)

Tesing: 500it [00:24, 20.42it/s]


In [65]:
from mmseg.core import pre_eval_to_metrics
ret_metrics=pre_eval_to_metrics(pre_eval_results,metrics=['mIoU'])
print(ret_metrics['IoU'].mean())

0.47525832
